Implementing matrix factorization on the netflix movie recommendation dataset

We will take the first thousand movies from the dataset and then see how many users we have have seen at least one of these movies

In [3]:
import os
import numpy as np


In [4]:
path_to_data = r"C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set"

first_thousand_movies = sorted([os.path.join(path_to_data, file) for file in os.listdir(path_to_data)])[:1000]

for i in range(10):
    print(first_thousand_movies[i])

C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000001.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000002.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000003.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000004.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000005.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000006.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000007.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000008.txt
C:\Users\Jackson Roubidoux\Continued_learning\data_science_notebooks\download\training_set\mv_0000009.txt
C:\Users\Jackson Roubidoux\Continued_learning\

Let's take a moment to understand how the matrix factorization works before we implement it with code.

Assume we have an *adjency matrix that represents users, movies and the ratings that users gave the movies. 

We will call this matrix A.

Let's take a small look at a section of A. We will take the first 10 movies from the dataset and select 2 people who watched each movie to get between 2 and 20 users and then we will build an adjency matrix. If the user didn't watch the movie then their entry in the adjency matrix will be denoted with NaN. 

In [11]:
movie_to_user_and_rating = {}
set_of_users = set()
movies = []

np.random.seed(42)
randomly_selected_10_movies = np.random.choice(first_thousand_movies, 10, replace=False)

for movie_file in randomly_selected_10_movies:
    with open(movie_file, 'r') as f_in:
        movie_id = next(f_in).strip().replace(":", "")
        movies.append(movie_id)
        movie_to_user_and_rating[movie_id] = {}

        rows = f_in.readlines()

        users = np.random.choice(rows, 2, replace=False)

        for user in users:
            user_id, rating, date = user.strip().split(",")
            set_of_users.add(user_id)
            if user_id in movie_to_user_and_rating[movie_id]:
                raise ValueError("This shouldn't be possible")
            else:
                movie_to_user_and_rating[movie_id][user_id] = rating

list_of_users = list(set_of_users)
A = np.full((len(list_of_users), len(movies)), np.nan)

movie_id_to_index_and_vice_versa = {}
user_id_to_index_and_vice_versa = {}

for i in range(len(movies)):
    movie_id_to_index_and_vice_versa[i] = movies[i]
    movie_id_to_index_and_vice_versa[movies[i]] = i

for i in range(len(list_of_users)):
    user_id_to_index_and_vice_versa[i] = list_of_users[i]
    user_id_to_index_and_vice_versa[list_of_users[i]] = i

for i, user_id in enumerate(list_of_users):
    for j, movie in enumerate(movies):
        if user_id in movie_to_user_and_rating[movie]:
            A[i][j] = movie_to_user_and_rating[movie][user_id]

print(A)


[[ 4. nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan  2. nan]
 [nan nan nan  2. nan nan nan nan nan nan]
 [nan  3. nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan  4.]
 [nan nan nan nan  4. nan nan nan nan nan]
 [nan nan nan nan nan nan  2. nan nan nan]
 [nan  4. nan nan nan nan nan nan nan nan]
 [ 3. nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan  4. nan nan nan nan]
 [nan nan nan  4. nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan  1.]
 [nan nan nan nan nan nan  3. nan nan nan]
 [nan nan nan nan nan nan nan  5. nan nan]
 [nan nan nan nan nan  3. nan nan nan nan]
 [nan nan nan nan nan nan nan nan  1. nan]
 [nan nan  3. nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan  4. nan nan]
 [nan nan nan nan  4. nan nan nan nan nan]
 [nan nan  4. nan nan nan nan nan nan nan]]


We see above a small represenation of our data in this matrix A. 

Each row in this matrix represents a user, the columns represent a movie, and the row-column values represent the rating a user has given a movie. In this small section we see that most users haven't rated most movies. This will trend true for the entire dataset. 

This issue is known as data sparsity and is a common problem for recommendation systems. Prior techniques for recommendation systems such as user-user collaborative filtering or item-item collaborative filtering can greatly suffer due to this issue. They also have other problems such as an inability to capture hidden relationships in the data to increase our ability to provide relavant recommendations. 

How do these less-than-ideal techniques work? They predict the rating a user would give for a given item based on the ratings similiar users gave that item. The same holds true for item-item collaborative filtering. 

WAIT A MINUTE!? You might be thinking: "So using UU CF (User-User Collaborative Filtering) or II CF (Item-Item Collaborative Filtering) depends heavily on how similar users are to other users (or items to other items) and you are trying to measure how similar they are using very little data? Data that is pretty sparse? (Since most users don't rate most movies, how can we determine intelligently which users or items are most similar. **Add some more intuition here**)"

The difficulty in comparing users or items due to sparse data and the inability of UU and II CF to capture hidden relationships drove the creation of matrix factorization techniques for recommendation systems. 

Also, if the above didn't make much sense, no stress. There are resources that outline the prior techniques in much more detail with better visuals. The take-away is this: older techniques for predicting what rating a user would give a product/item was limited because it couldn't learn complex/hidden relationships about users and products AND the lack of ratings by users made it difficult to predict a users rating on new products. Matrix factorization aims to resolve these issues. 




Alright, so MF (matrix factorization). The #$%@! is it and how does it resolve some issues for us?

We have x number of users, let's say we have 50 users and 10 movies that users can watch and rate. In our user-movie matrix (rows are users, columns are movies, entries in the matrix are ratings (or nothing if the user didn't rate the movie yet)) we would expect to have 500 entries. (50 users multiplied by 10 movies, which would be the total number of ratings). 

What we can do here is imagine two other matrices, we can call them P and Q and say that the result of multiplying P and Q gives us our matrix A. See the code below for a small visual. 

In [16]:
P = np.round(np.random.rand(20, 3), 2)
Q = np.round(np.random.rand(3, 10), 2)
A = np.round((P @ Q), 2)

print("Here is P: \n")
print(P)
print("\n\n\n")

print("Here is Q: \n")
print(Q)
print("\n\n\n")

print("Here is A, the result of multiplying P and Q together: \n")
print(A)

Here is P: 

[[0.18 0.3  0.05]
 [0.47 0.85 0.93]
 [0.93 0.13 0.38]
 [0.03 0.86 0.36]
 [0.63 0.71 0.43]
 [0.31 0.96 0.96]
 [0.29 0.48 0.97]
 [0.   0.41 0.1 ]
 [0.51 0.02 0.41]
 [0.22 0.53 1.  ]
 [0.8  0.85 0.64]
 [0.04 0.21 0.93]
 [0.37 0.1  0.7 ]
 [0.3  0.27 1.  ]
 [0.86 0.41 0.98]
 [0.65 0.74 0.03]
 [0.12 0.63 0.9 ]
 [0.62 0.13 0.12]
 [0.84 0.14 0.52]
 [0.04 0.08 0.67]]




Here is Q: 

[[0.69 0.34 0.22 0.81 0.22 0.54 0.97 0.15 0.94 0.18]
 [0.57 0.38 0.96 0.27 0.4  0.98 0.54 0.5  0.26 0.7 ]
 [0.01 0.05 0.6  0.34 0.95 0.9  0.71 0.82 0.67 0.13]]




Here is A, the result of multiplying P and Q together: 

[[0.3  0.18 0.36 0.24 0.21 0.44 0.37 0.22 0.28 0.25]
 [0.82 0.53 1.48 0.93 1.33 1.92 1.58 1.26 1.29 0.8 ]
 [0.72 0.38 0.56 0.92 0.62 0.97 1.24 0.52 1.16 0.31]
 [0.51 0.36 1.05 0.38 0.69 1.18 0.75 0.73 0.49 0.65]
 [0.84 0.51 1.08 0.85 0.83 1.42 1.3  0.8  1.06 0.67]
 [0.77 0.52 1.57 0.84 1.36 1.97 1.5  1.31 1.18 0.85]
 [0.48 0.33 1.11 0.69 1.18 1.5  1.23 1.08 1.05 0.51]
 [0.23 0.16 0.45 